In [ ]:
# running dbs using sql in python
# connecting db to sqlite
# executing question within sqlite and with python
# join different tables together to answer questions
# mongo is stretch 


curs2.close()

In [495]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3

In [496]:
df = pd.read_csv("titanic.csv")

In [497]:
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [498]:
df["SiblingsSpouses"] = df["Siblings/Spouses Aboard"]

In [499]:
df["ParentsChildren"] = df["Parents/Children Aboard"]

In [500]:
df = df.drop(["Parents/Children Aboard","Siblings/Spouses Aboard"], axis=1)

In [501]:
df.head(5)

,Survived,Pclass,Name,Sex,Age,Fare,SiblingsSpouses,ParentsChildren
0,0,3,Mr. Owen Harris Braund,male,22.0,7.2500,1,0
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,71.2833,1,0
2,1,3,Miss. Laina Heikkinen,female,26.0,7.9250,0,0
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,53.1000,1,0
4,0,3,Mr. William Henry Allen,male,35.0,8.0500,0,0


In [502]:
#Retreive length of each column 

def get_string_count(df):
    measurer = np.vectorize(len)
    m_list = measurer(df.values.astype(str)).max(axis=0)
    d = {}
    for i,column in enumerate(df):
        d.update({column:m_list[i]})
    s_count = pd.DataFrame(d, index = [0])
    return s_count

get_string_count(df)

,Survived,Pclass,Name,Sex,Age,Fare,SiblingsSpouses,ParentsChildren
0,1,1,81,6,4,8,1,1


In [503]:
#Possible schema to use

print(pd.io.sql.get_schema(df, 'Titanic'))

CREATE TABLE "Titanic" (
"Survived" INTEGER,
  "Pclass" INTEGER,
  "Name" TEXT,
  "Sex" TEXT,
  "Age" REAL,
  "Fare" REAL,
  "SiblingsSpouses" INTEGER,
  "ParentsChildren" INTEGER
)


In [504]:
sl_conn = sqlite3.connect('/Users/user/Documents/GitHub/Lambda/DS-Unit-3-Sprint-2-SQL-and-Databases/module3-nosql-and-document-oriented-databases/titanic.sqlite3')

In [ ]:
df.to_sql("titanic",con=sl_conn)

In [296]:
sl_curs = sl_conn.cursor()

In [506]:
sl_curs.execute("SELECT name FROM titanic LIMIT 10;").fetchall()

[('Mr. Owen Harris Braund',),
 ('Mrs. John Bradley (Florence Briggs Thayer) Cumings',),
 ('Miss. Laina Heikkinen',),
 ('Mrs. Jacques Heath (Lily May Peel) Futrelle',),
 ('Mr. William Henry Allen',),
 ('Mr. James Moran',),
 ('Mr. Timothy J McCarthy',),
 ('Master. Gosta Leonard Palsson',),
 ('Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson',),
 ('Mrs. Nicholas (Adele Achem) Nasser',)]

# Postgress Db Setup

In [507]:
import psycopg2
from creds import Elephant

In [535]:
dbname = Elephant.dbname #same as user
user = Elephant.user #same as dbname
password = Elephant.password #don't commit this to github!
host = Elephant.host #from SERVER type this in as string


pg_conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
pg_curs = pg_conn.cursor()
    

In [533]:
psy(pg_curs)

In [536]:
#Test connection was made properly

pg_curs.execute('SELECT * FROM test_table;')
pg_curs.fetchall()

[(2, 'a second row', {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})]

In [510]:
create_titanic_table = '''DROP TABLE IF EXISTS titanic4;
  CREATE TABLE titanic4 (
    index SERIAL PRIMARY KEY,
    Survived INT,
    Pclass INT,
    Name VARCHAR(81),
    Sex VARCHAR(6),
    Age FLOAT,
    Fare FLOAT,
    SiblingsSpouses INT,
    ParentsChildren INT
  );
'''

In [511]:
pg_curs.execute(create_titanic_table)

In [226]:
tuple(df.values[0])

(0, 0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 7.25, 1, 0)

In [537]:
example_insert = """
INSERT INTO titanic4
(Survived, Pclass, Name, Sex, Age, Fare, SiblingsSpouses, ParentsChildren)
VALUES """ + str(tuple(df.values[0])) + ';'"""
"""
print(example_insert)


INSERT INTO titanic4
(Survived, Pclass, Name, Sex, Age, Fare, SiblingsSpouses, ParentsChildren)
VALUES (0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 7.25, 1, 0);



In [512]:
# def insert_df_to_pg(df):

for i in df.values:
    passenger = str(tuple((i[0], i[1], i[2].replace("'", ""),
                           i[3], i[4], i[5], i[6], i[7])))
    insert = """
    INSERT INTO titanic4
    (Survived, Pclass, Name, Sex, Age, Fare, SiblingsSpouses, ParentsChildren)
    VALUES """ + passenger + ';' 
    pg_curs.execute(insert)

pg_curs.close()
pg_conn.commit()

In [538]:
pg_conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
pg_curs = pg_conn.cursor()
pg_curs.execute("SELECT * FROM titanic4 LIMIT 10;")

In [539]:
pg_curs.fetchall()

[(1, 0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 7.25, 1, 0),
 (2,
  1,
  1,
  'Mrs. John Bradley (Florence Briggs Thayer) Cumings',
  'female',
  38.0,
  71.2833,
  1,
  0),
 (3, 1, 3, 'Miss. Laina Heikkinen', 'female', 26.0, 7.925, 0, 0),
 (4,
  1,
  1,
  'Mrs. Jacques Heath (Lily May Peel) Futrelle',
  'female',
  35.0,
  53.1,
  1,
  0),
 (5, 0, 3, 'Mr. William Henry Allen', 'male', 35.0, 8.05, 0, 0),
 (6, 0, 3, 'Mr. James Moran', 'male', 27.0, 8.4583, 0, 0),
 (7, 0, 1, 'Mr. Timothy J McCarthy', 'male', 54.0, 51.8625, 0, 0),
 (8, 0, 3, 'Master. Gosta Leonard Palsson', 'male', 2.0, 21.075, 3, 1),
 (9,
  1,
  3,
  'Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson',
  'female',
  27.0,
  11.1333,
  0,
  2),
 (10,
  1,
  2,
  'Mrs. Nicholas (Adele Achem) Nasser',
  'female',
  14.0,
  30.0708,
  1,
  0)]

# Questions from Module 4

- How many passengers survived, and how many died?
- How many passengers were in each class?
- How many passengers survived/died within each class?
- What was the average age of survivors vs nonsurvivors?
- What was the average age of each passenger class?
- What was the average fare by passenger class? By survival?
- How many siblings/spouses aboard on average, by passenger class? By survival?
- How many parents/children aboard on average, by passenger class? By survival?
- Do any passengers have the same name?
- (Bonus! Hard, may require pulling and processing with Python) How many married
  couples were aboard the Titanic? Assume that two people (one `Mr.` and one
  `Mrs.`) with the same last name and with at least 1 sibling/spouse aboard are
  a married couple.

In [545]:
df.head(1)

,Survived,Pclass,Name,Sex,Age,Fare,SiblingsSpouses,ParentsChildren
0,0,3,Mr. Owen Harris Braund,male,22.0,7.25,1,0


In [ ]:
def psyco_q(query,session_name = pg_curs):
    #function to open new db session, run query, and close 
    pg_conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    pg_curs = pg_conn.cursor()
    pg_curs.execute(query)
    return pg_curs.fetchall()
    pg_curs.close()

# How many passengers survived, and how many died?
query1 = "SELECT count(Survived) FROM titanic4 WHERE Survived = 1;"
query2 = "SELECT count(Survived) FROM titanic4 WHERE Survived = 0;"
# How many passengers were in each class?
query3 = "SELECT Pclass,count(*) FROM titanic4 GROUP BY Pclass;"
# How many passengers survived/died within each class?
query4 = "SELECT Pclass,count(*) FROM titanic4 WHERE Survived = 1 GROUP BY Pclass;"
query5 = "SELECT Pclass,count(*) FROM titanic4 WHERE Survived = 0 GROUP BY Pclass;"
# What was the average age of survivors vs nonsurvivors?
query6 = "SELECT avg(Age) FROM titanic4 WHERE Survived = 1;"
query7 = "SELECT avg(Age) FROM titanic4 WHERE Survived = 0;"
# What was the average age of each passenger class?
query8 = "SELECT Pclass,avg(Age) FROM titanic4 WHERE Survived = 1 GROUP By Pclass;"
# What was the average fare by passenger class? By survival?
query9 = "SELECT Pclass,avg(Fare) FROM titanic4 GROUP BY Pclass;"
query10 = "SELECT Survived,avg(Fare) FROM titanic4 GROUP BY Survived;"
# How many siblings/spouses aboard on average, by passenger class? By survival?
query11 = "SELECT Pclass,avg(SiblingsSpouses) FROM titanic4 GROUP BY Pclass;"
query12 = "SELECT Survived,avg(SiblingsSpouses) FROM titanic4 GROUP BY Survived;"
# How many parents/children aboard on average, by passenger class? By survival?
query13 = "SELECT Pclass,avg(ParentsChildren) FROM titanic4 GROUP BY Pclass;"
query14 = "SELECT Survived,avg(ParentsChildren) FROM titanic4 GROUP BY Survived;"
# Do any passengers have the same name?
query15 = "SELECT Name,count(*) FROM titanic4 GROUP BY Name Having count(Name) > 1;"
# (Bonus! Hard, may require pulling and processing with Python) How many married couples were aboard the Titanic? 
# Assume that two people (one Mr. and one Mrs.) with the same last name and with at least 1 sibling/spouse aboard 
# are a married couple.
query16 = "SELECT Name FROM titanic4 WHERE Name = 'Mr. Owen Harris Braund';"

print(f'Question 1 - Survived:{psyco_q(query1)[0]} Died:{psyco_q(query2)[0]}')
print(f'Question 2 - {psyco_q(query3)}')
print(f'Question 3 - Survived(Per Class):{psyco_q(query4)} Died(Per Class):{psyco_q(query5)}')
print(f'Question 4 - Survivors:{psyco_q(query6)} Died:{psyco_q(query7)}')
print(f'Question 5 - Average Age Per Class:{psyco_q(query8)}')
print(f'Question 6 - Average Fare Per Class:{psyco_q(query9)} Average Fare per Survival:{psyco_q(query10)}')
print(f'Question 6 - Average SS Per Class:{psyco_q(query11)} Average SS per Survival:{psyco_q(query12)}')
print(f'Question 7 - Average PC Per Class:{psyco_q(query13)} Average PC per Survival:{psyco_q(query14)}')
print(f'Question 8 - Same Name:{psyco_q(query15)}')
print(f'Question 9 - Same Name:{(psyco_q(query16))}')


In [ ]:
query16 = "SELECT Name FROM titanic4 WHERE Name = 'Mr. Owen Harris Braund';"
query17 = "SELECT SUBSTRING_INDEX(Name,' ', 1) FROM titanic4;"
p = psyco_q(query17)
p
# print (',    ,'.join([str(i) for i in p]))
# print ([i for i in p])

In [654]:
df3 = df.copy()

In [662]:
last = []
for name in df3["Name"]:
    lastname = name.split()[-1]
    last.append(lastname)
df3["lastname"] = last


In [742]:
# [i for i in enumerate(df3["lastname"])] 

condition = (df3['lastname'].value_counts() >= 2).tolist()

len(df3['lastname'].value_counts())

# df3.loc[df3["SiblingsSpouses"] > 0 & df3["Name"].str.contains("Mrs.","Mr.")]

# df3.loc[df3["Name"].str.contains("Mrs.","Mr.")]


664

In [661]:
# df3["Name"].str.find(last)

TypeError: expected a string object, not list

# SQL Alchemy Imports

In [424]:
# Session = sessionmaker(bind=dest_db_con)
# session = Session()
# session.bulk_insert_mappings(MentorInformation, df.to_dict(orient="records"))
# session.close()

In [461]:
# df2 = pd.read_csv("titanic.csv")

In [478]:
# df2.head(5)

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [481]:
from sqlalchemy import create_engine
import psycopg2 as pg
from creds import Elephant
#load python script that batch loads pandas df to sql
from io import StringIO
import io

address = f'postgresql://{user}:{password}@{host}:5432/{dbname}'

engine = create_engine(address, max_overflow = -1)
connection = engine.raw_connection()
cursor = connection.cursor()

# pg_conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
# pg_curs = pg_conn.cursor()

# #df is the dataframe containing an index and the columns "Event" and "Day"
# #create Index column to use as primary key
# df2.reset_index(inplace=True)
# df2.rename(columns={'index':'Index'}, inplace =True)

#create the table but first drop if it already exists
command = '''DROP TABLE IF EXISTS titanic4;
  CREATE TABLE titanic4 (
    index SERIAL PRIMARY KEY,
    Survived INT,
    Pclass INT,
    Name VARCHAR(81),
    Sex VARCHAR(6),
    Age FLOAT,
    Fare FLOAT,
    SiblingsSpouses INT,
    ParentsChildren INT
  );
'''
cursor.execute(command)
connection.commit()
cursor.close()

#s

InvalidTextRepresentation: invalid input syntax for integer: "Mr. Owen Harris Braund"
CONTEXT:  COPY titanic4, line 1, column pclass: "Mr. Owen Harris Braund"
